In [ ]:
f_pre = []
f_avg = []
f_tru = []
prediction_list = []
average_list = []
true_list = []

for Rki in df_Rk.Rk:
    Rk = str(Rki)
    print Rk
    cmd_target_2015 = 'SELECT PTS,3P,TRB,AST,STL,BLK,TOV FROM NBA_player_data WHERE Player_Name IN (\'' + player + '\') AND Year IN (\'2015\') AND Rk < '+Rk+' ;'
    cmd_target_2014 = 'SELECT PTS,3P,TRB,AST,STL,BLK,TOV FROM NBA_player_data WHERE Player_Name IN (\'' + player + '\') AND Year IN (\'2014\') AND Rk >= '+Rk+' ;'
    cmd_train_2015 = 'SELECT Rk,Home_Away,DateDiff,TeamID,Win,OPPG,OTPR,O3Ppercent,ORPG,OBPG,OSPG,DEF,O3PM,OFGpercent,OTPG,OAPG,TPG,SPG,TRBR,OBLKpercent FROM NBA_player_data WHERE Player_Name IN (\'' + player + '\') AND Year IN (\'2015\') AND Rk < '+Rk+';'
    cmd_train_2014 = 'SELECT Rk,Home_Away,DateDiff,TeamID,Win,OPPG,OTPR,O3Ppercent,ORPG,OBPG,OSPG,DEF,O3PM,OFGpercent,OTPG,OAPG,TPG,SPG,TRBR,OBLKpercent FROM NBA_player_data WHERE Player_Name IN (\'' + player + '\') AND Year IN (\'2014\') AND Rk >= '+Rk+';'
    cmd_operate = 'SELECT Rk,Home_Away,DateDiff,TeamID,Win,OPPG,OTPR,O3Ppercent,ORPG,OBPG,OSPG,DEF,O3PM,OFGpercent,OTPG,OAPG,TPG,SPG,TRBR,OBLKpercent FROM NBA_player_data WHERE Player_Name IN (\'' + player + '\') AND Year IN (\'2015\') AND Rk = '+Rk+';'
    cmd_truth = 'SELECT PTS,3P,TRB,AST,STL,BLK,TOV FROM NBA_player_data WHERE Player_Name IN (\'' + player + '\') AND Year IN (\'2015\') AND Rk = '+Rk+' ;'

    df_target_2015 = pd.read_sql(cmd_target_2015, con=conn) 
    df_target_2014 = pd.read_sql(cmd_target_2014, con=conn) 
    df_train_2015 = pd.read_sql(cmd_train_2015, con=conn) 
    df_train_2014 = pd.read_sql(cmd_train_2014, con=conn) 
    df_operate = pd.read_sql(cmd_operate, con=conn) 
    df_truth = pd.read_sql(cmd_truth, con=conn) 
    df_truth = df_truth.applymap(lambda x: float(x))

    df_target=pd.concat([df_target_2014, df_target_2015],ignore_index=True)
    df_train=pd.concat([df_train_2014, df_train_2015],ignore_index=True)
    df_target = df_target.applymap(lambda x:float(x))
    df_train = df_train.applymap(lambda x:float(x))
    df_target_2015 = df_target_2015.applymap(lambda x: float(x))

    df_inquire = df_operate.applymap(lambda x:float(x))
    df_train_plus_inquire=pd.concat([df_train, df_inquire])
    df_raw = df_train_plus_inquire.reindex()
    df_raw_scaled = df_raw.copy()
    df_raw_scaled = df_raw_scaled.applymap(lambda x: np.log(x))
    df_raw_transform = df_raw.copy()

    df_raw_scaled = df_raw_scaled.apply(lambda x:preprocessing.StandardScaler().fit(x).transform(x))
    df_raw_transform = df_raw_transform.apply(lambda x:preprocessing.StandardScaler().fit(x))

    pca = PCA()
    pca.fit(df_raw_scaled)
    pca.n_components=7
    train_reduced = pca.fit_transform(df_raw_scaled)
    df_train_reduced=pd.DataFrame(train_reduced)
    df_evaluate = df_train_reduced.tail(1)
    df_train_scaled = df_train_reduced.iloc[:-1]


    # rf = RandomForestRegressor(n_estimators=100)
    # rf.fit(df_train_scaled, df_target)
    # predictions = rf.predict(df_evaluate).round()[0]

    PTS = LinR()
    PTS.fit(df_train_scaled, df_target.PTS)
    pPTS = PTS.predict(df_evaluate)
    REB = LinR()
    REB.fit(df_train_scaled, df_target.TRB)
    pREB = REB.predict(df_evaluate)
    AST = LinR()
    AST.fit(df_train_scaled, df_target.AST)
    pAST = AST.predict(df_evaluate)
    TP = LinR()
    TP.fit(df_train_scaled, df_target['3P'])
    pTP = TP.predict(df_evaluate)
    STL = LinR()
    STL.fit(df_train_scaled, df_target.STL)
    pSTL = STL.predict(df_evaluate)
    BLK = LinR()
    BLK.fit(df_train_scaled, df_target.BLK)
    pBLK = BLK.predict(df_evaluate)
    TOV = LinR()
    TOV.fit(df_train_scaled, df_target.TOV)
    pTOV = TOV.predict(df_evaluate)

    predictions = np.asarray([pPTS,pTP,pREB,pAST,pSTL,pBLK,pTOV])

    if int(Rk) < 20:
        average_stats=df_target.mean()
        #average_stats=df_target.mean().round()   
    elif int(Rk) >= 20:
        #average_stats=df_target_2015.mean().round()
        average_stats=df_target_2015.mean()
    average_stats=np.asarray(average_stats)
    true_stats = np.asarray(df_truth)[0]
    fanduel_pre, fanduel_avg, fanduel_tru = get_fanduel(predictions,average_stats,true_stats)
    f_pre.append(fanduel_pre)
    f_avg.append(fanduel_avg)
    f_tru.append(fanduel_tru)
    prediction_list.append(predictions)
    average_list.append(average_stats)
    true_list.append(true_stats)

score =[]
for i,f in enumerate(f_pre):
    s=get_trend(f,f_avg[i],f_tru[i])
    score.append(s)
a = [x for x in score if x != 2]

pts =[]
reb = []
ast = []
tp = []
stl =[]
blk =[]
to =[]
for i in range(len(predictions):
    p=get_trend(predictions[i][0],average_stats[i][0],true_stats[i][0])
    r=get_trend(predictions[i][2],average_stats[i][2],true_stats[i][2])
    a=get_trend(predictions[i][3],average_stats[i][3],true_stats[i][3])
    tpm=get_trend(predictions[i][1],average_stats[i][1],true_stats[i][1])
    s=get_trend(predictions[i][4],average_stats[i][4],true_stats[i][4])
    b=get_trend(predictions[i][5],average_stats[i][5],true_stats[i][5])
    t=get_trend(predictions[i][6],average_stats[i][6],true_stats[i][6])

    pts.append(p)
    reb.append(r)
    ast.append(a)
    tp.append(tpm)
    stl.append(s)
    blk.append(b)
    to.append(t)

pts = [x for x in pts if x != 2]
reb = [x for x in reb if x != 2]
ast = [x for x in ast if x != 2]
tp = [x for x in tp if x != 2]
stl = [x for x in stl if x != 2]
blk = [x for x in blk if x != 2]
to = [x for x in to if x != 2]


f_accuracy = a.count(1)/len(a)
pts_accuracy = pts.count(1)/len(pts)
reb_accuracy = reb.count(1)/len(reb)
ast_accuracy = ast.count(1)/len(ast)
tp_accuracy = tp.count(1)/len(tp)
stl_accuracy = stl.count(1)/len(stl)
blk_accuracy = blk.count(1)/len(blk)
to_accuracy = to.count(1)/len(to)